In [1]:
import glob
import itertools
import json
import os
import pandas
import scipy
from scipy import stats
import matplotlib.pyplot as plt


In [2]:
path = "ExperimentData"
dir_list = os.listdir(path)
 
print("Files and directories in '", path, "' :")
 
# prints all files
print(dir_list)

exp_paths = []

session_map = {}
for filename in dir_list:
    if filename.isdigit():
        exp_paths.append(path + "/" + filename + "/ResponsiveAR/Experiment/")
        #session_map[filename] = user_id
        files = os.listdir(path + "/" + filename + "/ResponsiveAR/Experiment/")
        name = files[0]
        index = name.find(".")
        user_id = name[index - 13: index]
        session_map[user_id] = int(filename)

print(exp_paths)

Files and directories in ' ExperimentData ' :
['61', '59', '68', '32', '35', '56', '51', '58', '60', '34', '33', '.DS_Store', '45', '44', '43', '36', '31', '53', '30', '37', '39', '46', '41', '83', '70', '71', '49', '40', '78']
['ExperimentData/61/ResponsiveAR/Experiment/', 'ExperimentData/59/ResponsiveAR/Experiment/', 'ExperimentData/68/ResponsiveAR/Experiment/', 'ExperimentData/32/ResponsiveAR/Experiment/', 'ExperimentData/35/ResponsiveAR/Experiment/', 'ExperimentData/56/ResponsiveAR/Experiment/', 'ExperimentData/51/ResponsiveAR/Experiment/', 'ExperimentData/58/ResponsiveAR/Experiment/', 'ExperimentData/60/ResponsiveAR/Experiment/', 'ExperimentData/34/ResponsiveAR/Experiment/', 'ExperimentData/33/ResponsiveAR/Experiment/', 'ExperimentData/45/ResponsiveAR/Experiment/', 'ExperimentData/44/ResponsiveAR/Experiment/', 'ExperimentData/43/ResponsiveAR/Experiment/', 'ExperimentData/36/ResponsiveAR/Experiment/', 'ExperimentData/31/ResponsiveAR/Experiment/', 'ExperimentData/53/ResponsiveAR/Exp

In [3]:
def parseFileNumber(fileName):
    fragments = fileName.split('_')
    return float(fragments[1])

In [4]:
responsiveData = []
experimentEvents = []

def parseFrame(frames, user_id):
    frames_list = []
    for frame in frames:
        frameData = {
            "userID": float,
            "frameNum": int,
            "unixTime": float,
            "timestamp": float,
            "hPos": {"x":float,"y":float,"z":float},
            "hDir": {"x":float,"y":float,"z":float},
            "hRot": {"i":float,"j":float,"k":float},
            "hAngl": {"x":float,"y":float,"z":float},
            "gazeOrigin": {"x":float,"y":float,"z":float},
            "gazeDirection": {"x":float,"y":float,"z":float},
            "rightHandRay": {"x":float,"y":float,"z":float},
            "leftHandRay": {"x":float,"y":float,"z":float},
            "experimentEvents": [],
            "responsiveData": []
        }
        responsiveEvent = frame["responsiveData"]
        experimentEvent = frame["experimentEvents"]
        frameData["userID"] = user_id
        frameData["frameNum"] = frame["frameNum"]
        frameData["unixTime"] = frame["unixTime"]
        frameData["timestamp"] = frame["timestamp"]
        frameData["hPos"] = frame["hPos"]
        frameData["hDir"] = frame["hDir"]
        frameData["hRot"] = frame["hRot"]
        frameData["hAngl"] = frame["hAngl"]
        frameData["gazeOrigin"] = frame["gazeOrigin"]
        frameData["gazeDirection"] = frame["gazeDirection"]
        frameData["rightHandRay"] = frame["rightHandRay"]
        frameData["leftHandRay"] = frame["leftHandRay"]
        frameData["experimentEvents"] = experimentEvent
        frameData["responsiveData"] = responsiveEvent
        frames_list.append(frameData)
        if responsiveEvent:
            responsiveData.append(responsiveEvent)
        if experimentEvent:
            experimentEvents.append(experimentEvent)
    return frames_list

def parseSession(session, user_id):
    sessionData = {
    "userID": float,
    "numFrames": int,
    "task": str,
    "start_time": float,
    "end_time": float,
    "total_time": float,
    "sessionNumber": int,
    "isResponsive": int,
    "frames": []
    }
    sessionData["userID"] = user_id
    sessionData["numFrames"] = session["numFrames"]
    sessionData["task"] = session["task"]
    sessionData["start_time"] = session["start_time"]
    sessionData["end_time"] = session["end_time"]
    sessionData["total_time"] = session["total_time"]
    sessionData["sessionNumber"] = session["sessionNumber"]
    sessionData["isResponsive"] = session["isResponsive"]
    return sessionData


In [5]:
data = []
frames = []
allsession = []
user_id = 0

for exp in exp_paths:
    experiment_data = {
    "path" : str,
    "sessions": {}
    }
    sessions = {}
    experiment = os.listdir(exp)
    current_path = exp
    experiment_data["path"] = current_path
    experiment.sort(key=parseFileNumber)
    for filename in experiment:
        if filename.endswith(".json"):
            # Prints only text file present in My Folder
            print(filename)
            firstFileFlag = False
            if("Session_0" in filename):
                firstFileFlag = True
            with open(current_path + filename, 'r') as f:
                json_data = json.load(f)
                if(firstFileFlag):
                    user_id = session_map[json_data["obj"]["userID"]]
                    session = parseSession(json_data["obj"]["sessionRecordings"][0], user_id)
                    frame = parseFrame(json_data["obj"]["sessionRecordings"][0]["frames"], user_id)
                    session["frames"] = frame
                    num = session["sessionNumber"]
                    sessions[num] = session
                    frames.append(frame)
                    allsession.append(session)
                else:
                    session = parseSession(json_data, user_id)
                    frame = parseFrame(json_data["frames"], user_id)
                    session["frames"] = frame
                    num = session["sessionNumber"]
                    sessions[num] = session
                    frames.append(frame)
                    allsession.append(session)
    experiment_data["sessions"] = sessions
    data.append(experiment_data)

#print(data)


ExperimentSession_0_1653325550021.json
ExperimentSession_1_1653325550021.json
ExperimentSession_2_1653325550021.json
ExperimentSession_3_1653325550021.json
ExperimentSession_4_1653325550021.json
ExperimentSession_5_1653325550021.json
ExperimentSession_6_1653325550021.json
ExperimentSession_7_1653325550021.json
ExperimentSession_8_1653325550021.json
ExperimentSession_9_1653325550021.json
ExperimentSession_0_1652910130460.json
ExperimentSession_1_1652910130460.json
ExperimentSession_2_1652910130460.json
ExperimentSession_3_1652910130460.json
ExperimentSession_4_1652910130460.json
ExperimentSession_5_1652910130460.json
ExperimentSession_6_1652910130460.json
ExperimentSession_7_1652910130460.json
ExperimentSession_8_1652910130460.json
ExperimentSession_9_1652910130460.json
ExperimentSession_0_1653072453927.json
ExperimentSession_1_1653072453927.json
ExperimentSession_2_1653072453927.json
ExperimentSession_3_1653072453927.json
ExperimentSession_4_1653072453927.json
ExperimentSession_5_16530

In [6]:
allsession

sessionsUnpacked = list(itertools.chain(allsession))
sessionsDf = pandas.DataFrame(sessionsUnpacked)
sessionsDf = sessionsDf.sort_values(by='sessionNumber')

sessionsDf = sessionsDf.loc[sessionsDf["sessionNumber"] < 9 ]
sessionsDf.head(1)

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames
0,61,92,The task is to click on the 2nd article title....,1653325572932,1653325575801,2869,0,True,"[{'userID': 61, 'frameNum': 1, 'unixTime': 165..."


USER 56 was unable to finish task 7 (when starting from 0) remove their task 8 from data

In [7]:
session_to_drop = sessionsDf.loc[(sessionsDf["userID"] == 56) & (sessionsDf["sessionNumber"] == 7)]
sessionsDf.drop(session_to_drop.index)

session_to_drop

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames
57,56,1434,The task is to find the current location on th...,1652899626599,1652899774967,148368,7,False,"[{'userID': 56, 'frameNum': 1741, 'unixTime': ..."


In [8]:
framesUnpacked = list(itertools.chain(*sessionsDf["frames"]))
framesDf = pandas.DataFrame(framesUnpacked)
print(framesDf.shape)
framesDf.head(1)

(88803, 14)


,userID,frameNum,unixTime,timestamp,hPos,hDir,hRot,hAngl,gazeOrigin,gazeDirection,rightHandRay,leftHandRay,experimentEvents,responsiveData
0,61,1,1653325566491,16470,"{'x': -0.018050827085971832, 'y': -0.004202843...","{'x': -0.0035658455453813076, 'y': -0.24633377...","{'x': 0.12413376569747925, 'y': -0.00124909367...","{'x': 14.260669708251953, 'y': 359.78918457031...","{'x': -0.017374971881508827, 'y': -0.004261824...","{'x': -0.0014919827226549387, 'y': -0.24794106...","{'x': -0.0683116614818573, 'y': 0.569282412528...","{'x': 0.0, 'y': 0.0, 'z': 0.0}","[{'unixTime': 1653325566402, 'systemTime': '10...","{'name': '', 'scale': {'x': 0.0, 'y': 0.0, 'z'..."


In [9]:
users = framesDf.loc[framesDf["experimentEvents"].str.len() != 0]
print(users.shape)

experimentUnpacked = list(itertools.chain(*users["experimentEvents"]))
experimentDf = pandas.DataFrame(experimentUnpacked)
experimentDf.head(1)
print(experimentDf.shape)
experimentDf.shape

experimentDf["userID"] = users["userID"]

for i in users.index:
    eventDF = pandas.DataFrame(users["experimentEvents"][i])

    for j in eventDF.index:
        time = eventDF["unixTime"][j]
        userID = users["userID"][i]
        mask = experimentDf["unixTime"] == time
        experimentDf.loc[mask, "userID"] = userID

experimentDf.head(1)

(1536, 14)
(2055, 10)


,unixTime,systemTime,isResponsive,eventName,task_number,task_type,object_scale,object_position,correct_answer,guess,userID
0,1653325566402,10-06-06-40,True,instruction,0,Article Display,"{'x': 0.25, 'y': 0.25, 'z': 0.25}","{'x': -0.01604645512998104, 'y': 0.02574505284...",2nd Title,,61.0


In [10]:
startEvents = experimentDf.loc[experimentDf["eventName"] == "start"]
startEvents.head(3)

,unixTime,systemTime,isResponsive,eventName,task_number,task_type,object_scale,object_position,correct_answer,guess,userID
1,1653325572982,10-06-12-98,True,start,0,Article Display,"{'x': 0.25, 'y': 0.25, 'z': 0.25}","{'x': -0.01604645512998104, 'y': 0.02574505284...",2nd Title,,61.0
3,1653088256965,16-10-56-96,False,start,0,Article Display,"{'x': 0.25, 'y': 0.25, 'z': 0.25}","{'x': 0.000685511389747262, 'y': 0.00223229080...",2nd Title,,46.0
5,1652829254176,16-14-14-17,True,start,0,Article Display,"{'x': 0.25, 'y': 0.25, 'z': 0.25}","{'x': 0.01994680054485798, 'y': -0.00246695242...",2nd Title,,39.0


In [11]:
sessionsDf

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames
0,61,92,The task is to click on the 2nd article title....,1653325572932,1653325575801,2869,0,True,"[{'userID': 61, 'frameNum': 1, 'unixTime': 165..."
200,46,229,The task is to click on the 2nd article title....,1653088256915,1653088273395,16480,0,False,"[{'userID': 46, 'frameNum': 1, 'unixTime': 165..."
190,39,220,The task is to click on the 2nd article title....,1652829254126,1652829262856,8730,0,True,"[{'userID': 39, 'frameNum': 1, 'unixTime': 165..."
260,40,627,The task is to click on the 2nd article title....,1652836605931,1652836653082,47151,0,False,"[{'userID': 40, 'frameNum': 1, 'unixTime': 165..."
40,35,159,The task is to click on the 2nd article title....,1652807292687,1652807301206,8519,0,True,"[{'userID': 35, 'frameNum': 1, 'unixTime': 165..."
...,...,...,...,...,...,...,...,...,...
158,31,164,The task is to find the low temperature on th...,1652726553097,1652726562238,9141,8,True,"[{'userID': 31, 'frameNum': 4994, 'unixTime': ..."
228,83,270,The task is to find the low temperature on th...,1653330386363,1653330408284,21921,8,True,"[{'userID': 83, 'frameNum': 2571, 'unixTime': ..."
148,36,174,The task is to find the low temperature on th...,1652812971891,1652812982716,10825,8,False,"[{'userID': 36, 'frameNum': 1424, 'unixTime': ..."
218,41,192,The task is to find the low temperature on th...,1652904828310,1652904841951,13641,8,True,"[{'userID': 41, 'frameNum': 2172, 'unixTime': ..."


In [12]:
sessionsDf["Actual_Start_Time"] = sessionsDf["start_time"]

for i in startEvents.index:
    userID = startEvents["userID"][i]
    task_num = startEvents["task_number"][i]
    mask = (sessionsDf["userID"] == userID) & (sessionsDf["sessionNumber"] == task_num)
    sessionsDf.loc[mask, "Actual_Start_Time"] = startEvents["unixTime"][i]

sessionsDf.head(1)

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames,Actual_Start_Time
0,61,92,The task is to click on the 2nd article title....,1653325572932,1653325575801,2869,0,True,"[{'userID': 61, 'frameNum': 1, 'unixTime': 165...",1653325572982


In [13]:
endEvents = experimentDf.loc[experimentDf["eventName"] == "complete"]
endEvents.head(1)

,unixTime,systemTime,isResponsive,eventName,task_number,task_type,object_scale,object_position,correct_answer,guess,userID
59,1653347142364,16-05-42-36,True,complete,0,Article Display,"{'x': 0.25, 'y': 0.25, 'z': 0.25}","{'x': 0.021247876808047295, 'y': 0.05604220554...",2nd Title,2nd Title,53.0


In [14]:
sessionsDf["Actual_End_Time"] = sessionsDf["end_time"]

for i in endEvents.index:
    userID = endEvents["userID"][i]
    task_num = endEvents["task_number"][i]
    mask = (sessionsDf["userID"] == userID) & (sessionsDf["sessionNumber"] == task_num)
    sessionsDf.loc[mask, "Actual_End_Time"] = endEvents["unixTime"][i]

sessionsDf.head(1)

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames,Actual_Start_Time,Actual_End_Time
0,61,92,The task is to click on the 2nd article title....,1653325572932,1653325575801,2869,0,True,"[{'userID': 61, 'frameNum': 1, 'unixTime': 165...",1653325572982,1653325575801


In [15]:
end_time_diff = sessionsDf["end_time"].to_numpy() - sessionsDf["Actual_End_Time"].to_numpy()
end_time_diff

array([  0, -11,   0,  -5, -11,   0,   0, -10,   0, -11,  -4,   0, -10,
       -11, -10, -10,   0, -10, -10, -10,   0,   0,   0,   0,   0,   0,
         0,   0,  -9,   0,   0,   0,   0,   0,   0, -10,   0,   0,   0,
       -10,   0,   0, -11,   0, -10,   0, -10, -10,   0,   0,   0, -10,
       -10,  -2, -10, -10, -11, -10,  -4,   0,   0,   0,   0,   0,   0,
        -9,   0,   0,   0,   0,  -9,  -8,  -6, -10,   0, -10, -10,   0,
         0,   0,   0,   0, -10,   0, -11,   0, -10, -10,   0,   0,   0,
         0,  -7, -10,   0,   0, -10,   0,  -4, -10, -10, -10, -10, -10,
        -9,  -6,  -9, -10,   0,   0,   0,  -9,   0, -10,  -5, -10, -10,
        -7,   0,   0,   0,   0, -11, -10,  -8,   0,   0,  -5,   0,   0,
         0,  -9,   0,  -3,   0,   0,  -9,   0,   0,   0,  -5,   0,  -9,
         0, -10,   0,  -9,   0,  -5,   0,  -9,   0,   0,  -9,  -5,   0,
         0,   0,   0, -11,  -9,   0,   0,   0,   0,   0,   0,   0, -10,
         0,  -9,   0,  -5,   0,   0, -10, -10,   0,   0, -15, -1

In [16]:
guessEvents = experimentDf.loc[experimentDf["eventName"] == "answer"]
guessEvents.head(3)

sessionsDf["Guesses"] = 0

In [17]:
for i in guessEvents.index:
    userID = guessEvents["userID"][i]
    task_num = guessEvents["task_number"][i]
    mask = (sessionsDf["userID"] == userID) & (sessionsDf["sessionNumber"] == task_num)
    sessionsDf.loc[mask, "Guesses"] += 1

In [18]:
start_time_diff = sessionsDf["start_time"].to_numpy() - sessionsDf["Actual_Start_Time"].to_numpy()
start_time_diff

array([-50, -50, -50, -60, -50, -50, -45, -59, -50, -56, -60, -50, -50,
       -61, -60, -51, -49, -60, -60, -60, -50, -60, -50, -60, -56, -50,
       -49, -50, -40, -39, -30, -41, -39, -51, -40, -40, -45, -40, -40,
       -43, -40, -40, -40, -40, -40, -40, -28, -40, -40, -46, -40, -40,
       -50, -40, -40, -30, -40, -40, -30, -40, -40, -40, -40, -40, -41,
       -40, -40, -40, -41, -39, -40, -46, -41, -41, -40, -41, -40, -39,
       -41, -39, -40, -40, -40, -41, -50, -40, -40, -30, -40, -40, -40,
       -36, -41, -39, -37, -40, -39, -40, -40, -40, -40, -40, -41, -41,
       -46, -40, -40, -40, -41, -45, -40, -47, -10, -19, -21, -10, -20,
       -16, -30, -19, -20, -20, -20, -20, -22, -21, -10, -10, -19, -20,
       -19, -21, -20, -21, -20, -19, -10, -20, -21, -19,   0,   0, -10,
       -11,   0, -10, -10, -10, -10,   0, -10,  -4,   0,   0, -11,   0,
       -10, -10, -10, -10, -10, -11,  -6,  -4, -10, -10, -10,  -3, -19,
       -11, -10, -10, -10, -10, -16, -10,  -8, -10, -11, -20, -1

In [19]:
sessionsDf["Actual_Total_Time"] = sessionsDf["Actual_End_Time"].to_numpy() - sessionsDf["Actual_Start_Time"].to_numpy()
sessionsDf.head(1)

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames,Actual_Start_Time,Actual_End_Time,Guesses,Actual_Total_Time
0,61,92,The task is to click on the 2nd article title....,1653325572932,1653325575801,2869,0,True,"[{'userID': 61, 'frameNum': 1, 'unixTime': 165...",1653325572982,1653325575801,1,2819


In [20]:
start_time_diff = sessionsDf["total_time"].to_numpy() - sessionsDf["Actual_Total_Time"].to_numpy()
start_time_diff

array([ 50,  39,  50,  55,  39,  50,  45,  49,  50,  45,  56,  50,  40,
        50,  50,  41,  49,  50,  50,  50,  50,  60,  50,  60,  56,  50,
        49,  50,  31,  39,  30,  41,  39,  51,  40,  30,  45,  40,  40,
        33,  40,  40,  29,  40,  30,  40,  18,  30,  40,  46,  40,  30,
        40,  38,  30,  20,  29,  30,  26,  40,  40,  40,  40,  40,  41,
        31,  40,  40,  41,  39,  31,  38,  35,  31,  40,  31,  30,  39,
        41,  39,  40,  40,  30,  41,  39,  40,  30,  20,  40,  40,  40,
        36,  34,  29,  37,  40,  29,  40,  36,  30,  30,  30,  31,  31,
        37,  34,  31,  30,  41,  45,  40,  38,  10,   9,  16,   0,  10,
         9,  30,  19,  20,  20,   9,  10,  14,  21,  10,   5,  19,  20,
        19,  12,  20,  18,  20,  19,   1,  20,  21,  19,  -5,   0,   1,
        11, -10,  10,   1,  10,   5,   0,   1,   4,   0,  -9,   6,   0,
        10,  10,  10,  -1,   1,  11,   6,   4,  10,  10,  10,   3,   9,
        11,   1,  10,   5,  10,  16,   0,  -2,  10,  11,   5,  -

In [21]:
responsiveDf = sessionsDf.loc[sessionsDf["isResponsive"] == True]
responsiveDf = responsiveDf.sort_values("Actual_Total_Time")
responsiveDf

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames,Actual_Start_Time,Actual_End_Time,Guesses,Actual_Total_Time
101,33,53,The task is to click the 5th article title.\n\...,1652743242306,1652743244466,2160,1,True,"[{'userID': 33, 'frameNum': 95, 'unixTime': 16...",1652743242346,1652743244466,1,2120
181,37,44,The task is to click the 5th article title.\n\...,1652818011143,1652818013353,2210,1,True,"[{'userID': 37, 'frameNum': 53, 'unixTime': 16...",1652818011183,1652818013353,1,2170
151,31,67,The task is to click the 5th article title.\n\...,1652726021550,1652726023839,2289,1,True,"[{'userID': 31, 'frameNum': 189, 'unixTime': 1...",1652726021589,1652726023839,1,2250
1,61,48,The task is to click the 5th article title.\n\...,1653325578382,1653325580702,2320,1,True,"[{'userID': 61, 'frameNum': 93, 'unixTime': 16...",1653325578422,1653325580702,1,2280
163,53,57,The task is to click the Date of the 1st artic...,1653347166484,1653347168964,2480,3,True,"[{'userID': 53, 'frameNum': 361, 'unixTime': 1...",1653347166514,1653347168974,1,2460
...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,31,1467,The task is to find the current location on th...,1652726388185,1652726544687,156502,7,True,"[{'userID': 31, 'frameNum': 3527, 'unixTime': ...",1652726388185,1652726544687,24,156502
155,31,1719,The task is to type phone number 805-UCSB-EDU ...,1652726084621,1652726272953,188332,5,True,"[{'userID': 31, 'frameNum': 806, 'unixTime': 1...",1652726084621,1652726272953,4,188332
166,53,1921,The task is to find the current temperature on...,1653347322034,1653347533250,211216,6,True,"[{'userID': 53, 'frameNum': 1803, 'unixTime': ...",1653347322037,1653347533250,23,211213
135,43,2221,The task is to type phone number 805-UCSB-EDU ...,1652990824367,1652991077151,252784,5,True,"[{'userID': 43, 'frameNum': 1430, 'unixTime': ...",1652990824377,1652991077162,3,252785


In [22]:
nonresponsiveDf = sessionsDf.loc[sessionsDf["isResponsive"] == False]
nonresponsiveDf = nonresponsiveDf.sort_values("Actual_Total_Time")
nonresponsiveDf

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames,Actual_Start_Time,Actual_End_Time,Guesses,Actual_Total_Time
32,32,76,The task is to click the Author of the 4th art...,1652736991131,1652736994072,2941,2,False,"[{'userID': 32, 'frameNum': 223, 'unixTime': 1...",1652736991161,1652736994076,1,2915
202,46,77,The task is to click the Author of the 4th art...,1653088327966,1653088331007,3041,2,False,"[{'userID': 46, 'frameNum': 715, 'unixTime': 1...",1653088328006,1653088331007,1,3001
122,44,100,The task is to click the Author of the 4th art...,1652995937411,1652995940511,3100,2,False,"[{'userID': 44, 'frameNum': 340, 'unixTime': 1...",1652995937451,1652995940511,1,3060
140,36,120,The task is to click on the 2nd article title....,1652812829574,1652812832864,3290,0,False,"[{'userID': 36, 'frameNum': 1, 'unixTime': 165...",1652812829624,1652812832874,1,3250
80,60,103,The task is to click on the 2nd article title....,1653093121501,1653093124821,3320,0,False,"[{'userID': 60, 'frameNum': 1, 'unixTime': 165...",1653093121561,1653093124831,1,3270
...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,46,722,The task is to find the current location on th...,1653088505254,1653088645361,140107,7,False,"[{'userID': 46, 'frameNum': 1682, 'unixTime': ...",1653088505259,1653088645367,37,140108
57,56,1434,The task is to find the current location on th...,1652899626599,1652899774967,148368,7,False,"[{'userID': 56, 'frameNum': 1741, 'unixTime': ...",1652899626604,1652899774967,45,148363
98,34,1464,The task is to find the low temperature on th...,1652747802011,1652747952843,150832,8,False,"[{'userID': 34, 'frameNum': 2625, 'unixTime': ...",1652747802021,1652747952843,45,150822
88,60,2011,The task is to find the low temperature on th...,1653093267183,1653093491077,223894,8,False,"[{'userID': 60, 'frameNum': 1445, 'unixTime': ...",1653093267194,1653093491077,55,223883


In [62]:
#task 0
i = 0
taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
taskDf
task_r = taskDf.loc[taskDf["isResponsive"] == True]
task_n = taskDf.loc[taskDf["isResponsive"] == False]

print(f"TASK {i}: " + task_r["task"][i])

taskRAvgTime = task_r['Actual_Total_Time'].mean()
taskRStd = task_r['Actual_Total_Time'].std()
print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Actual_Total_Time'].mean()
taskNStd = task_n['Actual_Total_Time'].std()
print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Actual_Total_Time'], task_n['Actual_Total_Time']))


TASK 0: The task is to click on the 2nd article title.

When you are ready, click the 'Start Task' button.
responsive task 0 Time: 5594.800 ± 2071.136
normal task 0 Time: 10686.077 ± 12046.905
Ttest_indResult(statistic=-1.6140752133417882, pvalue=0.11858374402699252)


In [63]:
#Task 1
i = 1
taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
task_r = taskDf.loc[taskDf["isResponsive"] == True]
task_n = taskDf.loc[taskDf["isResponsive"] == False]

print(f"TASK {i}: " + task_r["task"][i])

taskRAvgTime = task_r['Actual_Total_Time'].mean()
taskRStd = task_r['Actual_Total_Time'].std()
print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Actual_Total_Time'].mean()
taskNStd = task_n['Actual_Total_Time'].std()
print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Actual_Total_Time'], task_n['Actual_Total_Time']))

TASK 1: The task is to click the 5th article title.

When you are ready, click the 'Start Task' button.
responsive task 1 Time: 4410.000 ± 2979.387
normal task 1 Time: 17486.769 ± 15627.594
Ttest_indResult(statistic=-3.1836436413663636, pvalue=0.003751697642330371)


In [64]:
#Task 2
i = 2
taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
taskDf
task_r = taskDf.loc[taskDf["isResponsive"] == True]
task_n = taskDf.loc[taskDf["isResponsive"] == False]

print(f"TASK {i}: " + task_r["task"][i])

taskRAvgTime = task_r['Actual_Total_Time'].mean()
taskRStd = task_r['Actual_Total_Time'].std()
print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Actual_Total_Time'].mean()
taskNStd = task_n['Actual_Total_Time'].std()
print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Actual_Total_Time'], task_n['Actual_Total_Time']))

TASK 2: The task is to click the Author of the 4th article.

When you are ready, click the 'Start Task' button.
responsive task 2 Time: 13756.333 ± 20647.850
normal task 2 Time: 11130.308 ± 7523.474
Ttest_indResult(statistic=0.43339266165986684, pvalue=0.6683040884349696)


In [65]:
#task 3
i = 3
taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
taskDf
task_r = taskDf.loc[taskDf["isResponsive"] == True]
task_n = taskDf.loc[taskDf["isResponsive"] == False]

print(f"TASK {i}: " + task_r["task"][i])

taskRAvgTime = task_r['Actual_Total_Time'].mean()
taskRStd = task_r['Actual_Total_Time'].std()
print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Actual_Total_Time'].mean()
taskNStd = task_n['Actual_Total_Time'].std()
print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Actual_Total_Time'], task_n['Actual_Total_Time']))

TASK 3: The task is to click the Date of the 1st article.

When you are ready, click the 'Start Task' button.
responsive task 3 Time: 6971.733 ± 5768.137
normal task 3 Time: 10985.462 ± 6058.189
Ttest_indResult(statistic=-1.794139530119122, pvalue=0.0844290233895711)


In [66]:
#task 4
i = 4
taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
taskDf
task_r = taskDf.loc[taskDf["isResponsive"] == True]
task_n = taskDf.loc[taskDf["isResponsive"] == False]

print(f"TASK {i}: " + task_r["task"][i])

taskRAvgTime = task_r['Actual_Total_Time'].mean()
taskRStd = task_r['Actual_Total_Time'].std()
print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Actual_Total_Time'].mean()
taskNStd = task_n['Actual_Total_Time'].std()
print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Actual_Total_Time'], task_n['Actual_Total_Time']))

TASK 4: The task is to type the phone number 949-927-4580 on the touchpad then click call. 

When you are ready, click the 'Start Task' button.
responsive task 4 Time: 26987.933 ± 11852.120
normal task 4 Time: 24613.154 ± 8343.192
Ttest_indResult(statistic=0.6036984084740925, pvalue=0.5512754025277893)


In [67]:
#task 5
i = 5
taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
taskDf
task_r = taskDf.loc[taskDf["isResponsive"] == True]
task_n = taskDf.loc[taskDf["isResponsive"] == False]

print(f"TASK {i}: " + task_r["task"][i])

taskRAvgTime = task_r['Actual_Total_Time'].mean()
taskRStd = task_r['Actual_Total_Time'].std()
print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Actual_Total_Time'].mean()
taskNStd = task_n['Actual_Total_Time'].std()
print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Actual_Total_Time'], task_n['Actual_Total_Time']))

TASK 5: The task is to type phone number 805-UCSB-EDU on the touchpad then click Call. 

When you are ready, click the 'Start Task' button.
responsive task 5 Time: 54778.533 ± 69352.962
normal task 5 Time: 22833.923 ± 9126.820
Ttest_indResult(statistic=1.6443482550230133, pvalue=0.11214325087648128)


In [68]:
#task 6
i = 6
taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
taskDf
task_r = taskDf.loc[taskDf["isResponsive"] == True]
task_n = taskDf.loc[taskDf["isResponsive"] == False]

print(f"TASK {i}: " + task_r["task"][i])

taskRAvgTime = task_r['Actual_Total_Time'].mean()
taskRStd = task_r['Actual_Total_Time'].std()
print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Actual_Total_Time'].mean()
taskNStd = task_n['Actual_Total_Time'].std()
print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Actual_Total_Time'], task_n['Actual_Total_Time']))

TASK 6: The task is to find the current temperature on the weather application and type it into the keyboard. 

When you are ready, click the 'Start Task' button.
responsive task 6 Time: 72233.133 ± 107969.356
normal task 6 Time: 22913.846 ± 9993.546
Ttest_indResult(statistic=1.6367730840786845, pvalue=0.11372684168163424)


In [69]:
#task 7
i = 7
taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
taskDf
task_r = taskDf.loc[taskDf["isResponsive"] == True]
task_n = taskDf.loc[taskDf["isResponsive"] == False]

print(f"TASK {i}: " + task_r["task"][i])

taskRAvgTime = task_r['Actual_Total_Time'].mean()
taskRStd = task_r['Actual_Total_Time'].std()
print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Actual_Total_Time'].mean()
taskNStd = task_n['Actual_Total_Time'].std()
print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Actual_Total_Time'], task_n['Actual_Total_Time']))

TASK 7: The task is to find the current location on the weather application and type it into the keyboard. The answer is not case sensitive.

When you are ready, click the 'Start Task' button.
responsive task 7 Time: 65005.400 ± 34666.506
normal task 7 Time: 72177.000 ± 88483.062
Ttest_indResult(statistic=-0.2899470228404043, pvalue=0.774156628061639)


In [70]:
#task 8
i = 8
taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
taskDf
task_r = taskDf.loc[taskDf["isResponsive"] == True]
task_n = taskDf.loc[taskDf["isResponsive"] == False]

print(f"TASK {i}: " + task_r["task"][i])

taskRAvgTime = task_r['Actual_Total_Time'].mean()
taskRStd = task_r['Actual_Total_Time'].std()
print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Actual_Total_Time'].mean()
taskNStd = task_n['Actual_Total_Time'].std()
print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Actual_Total_Time'], task_n['Actual_Total_Time']))

TASK 8: The task is to find the low temperature  on the weather application and type it into the keyboard. 

When you are ready, click the 'Start Task' button.
responsive task 8 Time: 35972.200 ± 37472.014
normal task 8 Time: 48980.615 ± 65756.934
Ttest_indResult(statistic=-0.6544210060544706, pvalue=0.5185881523790975)


In [71]:
#each task in a loop
for i in range(0,9):
    taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
    taskDf
    task_r = taskDf.loc[taskDf["isResponsive"] == True]
    task_n = taskDf.loc[taskDf["isResponsive"] == False]

    print(f"TASK {i}: " + task_r["task"][i])

    taskRAvgTime = task_r['Actual_Total_Time'].mean()
    taskRStd = task_r['Actual_Total_Time'].std()
    print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

    taskNAvgTime = task_n['Actual_Total_Time'].mean()
    taskNStd = task_n['Actual_Total_Time'].std()
    print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

    print(scipy.stats.ttest_ind(task_r['Actual_Total_Time'], task_n['Actual_Total_Time']))

TASK 0: The task is to click on the 2nd article title.

When you are ready, click the 'Start Task' button.
responsive task 0 Time: 5594.800 ± 2071.136
normal task 0 Time: 10686.077 ± 12046.905
Ttest_indResult(statistic=-1.6140752133417882, pvalue=0.11858374402699252)
TASK 1: The task is to click the 5th article title.

When you are ready, click the 'Start Task' button.
responsive task 1 Time: 4410.000 ± 2979.387
normal task 1 Time: 17486.769 ± 15627.594
Ttest_indResult(statistic=-3.1836436413663636, pvalue=0.003751697642330371)
TASK 2: The task is to click the Author of the 4th article.

When you are ready, click the 'Start Task' button.
responsive task 2 Time: 13756.333 ± 20647.850
normal task 2 Time: 11130.308 ± 7523.474
Ttest_indResult(statistic=0.43339266165986684, pvalue=0.6683040884349696)
TASK 3: The task is to click the Date of the 1st article.

When you are ready, click the 'Start Task' button.
responsive task 3 Time: 6971.733 ± 5768.137
normal task 3 Time: 10985.462 ± 6058.18

In [33]:
for i in range(0,9):
    print(f"TASK {str(i)}:")
    taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
    taskDf
    task_r = taskDf.loc[taskDf["isResponsive"] == True]
    task_n = taskDf.loc[taskDf["isResponsive"] == False]

    taskRAvgTime = task_r['Guesses'].mean()
    taskRStd = task_r['Guesses'].std()
    print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

    taskNAvgTime = task_n['Guesses'].mean()
    taskNStd = task_n['Guesses'].std()
    print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

    print(scipy.stats.ttest_ind(task_r['Guesses'], task_n['Guesses']))

TASK 0:
responsive task 0 Time: 1.067 ± 0.258
normal task 0 Time: 1.077 ± 0.277
Ttest_indResult(statistic=-0.10129391775555853, pvalue=0.9200942656643553)
TASK 1:
responsive task 1 Time: 1.067 ± 0.258
normal task 1 Time: 1.692 ± 0.947
Ttest_indResult(statistic=-2.460988979756418, pvalue=0.020810281168403156)
TASK 2:
responsive task 2 Time: 1.800 ± 2.077
normal task 2 Time: 2.000 ± 1.871
Ttest_indResult(statistic=-0.26595305372511563, pvalue=0.792373006600172)
TASK 3:
responsive task 3 Time: 1.133 ± 0.352
normal task 3 Time: 1.538 ± 0.967
Ttest_indResult(statistic=-1.5140709079451053, pvalue=0.14207073219020286)
TASK 4:
responsive task 4 Time: 1.067 ± 0.258
normal task 4 Time: 1.077 ± 0.277
Ttest_indResult(statistic=-0.10129391775555853, pvalue=0.9200942656643553)
TASK 5:
responsive task 5 Time: 1.400 ± 0.910
normal task 5 Time: 1.000 ± 0.000
Ttest_indResult(statistic=1.5803597520225598, pvalue=0.12611480681378756)
TASK 6:
responsive task 6 Time: 10.333 ± 14.024
normal task 6 Time: 5.00

In [34]:
usersDf = sessionsDf[["userID"]]
usersDf = usersDf.drop_duplicates()
usersDf = usersDf.sort_values("userID")
usersDf

,userID
170,30
150,31
30,32
100,33
90,34
40,35
140,36
180,37
190,39
260,40


In [35]:
allTasksDf = usersDf

In [36]:
for i in range(0,9):
    allTasksDf[f"task{i}"] = i

allTasksDf["all_tasks"] = 0
allTasksDf["group"] = "A"
allTasksDf

,userID,task0,task1,task2,task3,task4,task5,task6,task7,task8,all_tasks,group
170,30,0,1,2,3,4,5,6,7,8,0,A
150,31,0,1,2,3,4,5,6,7,8,0,A
30,32,0,1,2,3,4,5,6,7,8,0,A
100,33,0,1,2,3,4,5,6,7,8,0,A
90,34,0,1,2,3,4,5,6,7,8,0,A
40,35,0,1,2,3,4,5,6,7,8,0,A
140,36,0,1,2,3,4,5,6,7,8,0,A
180,37,0,1,2,3,4,5,6,7,8,0,A
190,39,0,1,2,3,4,5,6,7,8,0,A
260,40,0,1,2,3,4,5,6,7,8,0,A


In [37]:
for i in allTasksDf.index:
    user = allTasksDf["userID"][i]
    userSessions = sessionsDf.loc[sessionsDf["userID"] == user]
    total = 0
    task = userSessions.loc[userSessions["sessionNumber"] == 0]
    allTasksDf.loc[i, "task0"] = task["Actual_Total_Time"].to_numpy()[0]
    total = int(task["Actual_Total_Time"])
    task = userSessions.loc[userSessions["sessionNumber"] == 1]
    allTasksDf.loc[i, "task1"] = task["Actual_Total_Time"].to_numpy()[0]
    total += int(task["Actual_Total_Time"])
    task = userSessions.loc[userSessions["sessionNumber"] == 2]
    allTasksDf.loc[i, "task2"] = task["Actual_Total_Time"].to_numpy()[0]
    total += int(task["Actual_Total_Time"])
    task = userSessions.loc[userSessions["sessionNumber"] == 3]
    allTasksDf.loc[i, "task3"] = task["Actual_Total_Time"].to_numpy()[0]
    total += int(task["Actual_Total_Time"])
    task = userSessions.loc[userSessions["sessionNumber"] == 4]
    allTasksDf.loc[i, "task4"] = task["Actual_Total_Time"].to_numpy()[0]
    total += int(task["Actual_Total_Time"])
    task = userSessions.loc[userSessions["sessionNumber"] == 5]
    allTasksDf.loc[i, "task5"] = task["Actual_Total_Time"].to_numpy()[0]
    total += int(task["Actual_Total_Time"])
    task = userSessions.loc[userSessions["sessionNumber"] == 6]
    allTasksDf.loc[i, "task6"] = task["Actual_Total_Time"].to_numpy()[0]
    total += int(task["Actual_Total_Time"])
    task = userSessions.loc[userSessions["sessionNumber"] == 7]
    allTasksDf.loc[i, "task7"] = task["Actual_Total_Time"].to_numpy()[0]
    total += int(task["Actual_Total_Time"])
    task = userSessions.loc[userSessions["sessionNumber"] == 8]
    allTasksDf.loc[i, "task8"] = task["Actual_Total_Time"].to_numpy()[0]
    total += int(task["Actual_Total_Time"])
    allTasksDf.loc[i, "all_tasks"] = total
    if(task["isResponsive"].to_numpy()[0] == True):
        allTasksDf.loc[i, "group"] = "B"

#allTasksDf = allTasksDf.drop(columns=["task{i}"])

In [38]:
allTasksDf = allTasksDf.sort_values("all_tasks")
allTasksDf

,userID,task0,task1,task2,task3,task4,task5,task6,task7,task8,all_tasks,group
0,61,2819,2280,2750,2983,18900,13570,10451,32670,10160,96583,B
140,36,3250,7150,9881,5910,13390,12400,17100,17020,10820,96921,A
30,32,3994,4471,2915,7051,16741,28440,10321,39910,7950,121793,A
230,70,4574,10060,12190,6031,25960,15810,18989,27610,10460,131684,A
70,58,4340,11560,11071,16220,22540,38590,20421,20420,10928,156090,A
170,30,17191,12730,17180,9970,21701,16280,16921,41531,9373,162877,A
100,33,4600,2120,6819,4950,21896,17790,11970,54381,55260,179786,B
20,68,5242,8141,15202,4435,14831,13650,32120,38646,49680,181947,A
210,41,3201,6310,4120,6479,36301,26860,15320,72051,13641,184283,B
270,78,9690,8510,27099,5949,32371,33200,25280,21980,28561,192640,A


In [39]:
groupA = allTasksDf.loc[allTasksDf["group"] == "A"]
groupB = allTasksDf.loc[allTasksDf["group"] == "B"]

groupATime = groupA['all_tasks'].mean()
groupAStd = groupA['all_tasks'].std()
print(f"group A {str(i)} Time: {groupATime:.3f} ± {groupAStd:.3f}")

groupBTime = groupB['all_tasks'].mean()
groupBStd = groupB['all_tasks'].std()
print(f"group B {str(i)} Time: {groupBTime:.3f} ± {groupBStd:.3f}")

print(scipy.stats.ttest_ind(groupB['all_tasks'], groupA['all_tasks']))

group A 220 Time: 241807.154 ± 128591.865
group B 220 Time: 285710.067 ± 135994.497
Ttest_indResult(statistic=0.8735591457095271, pvalue=0.39035561499718885)


In [40]:
sessionsDf.loc[(sessionsDf["userID"] == 30) & (sessionsDf["sessionNumber"] == 0)]

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames,Actual_Start_Time,Actual_End_Time,Guesses,Actual_Total_Time
170,30,240,The task is to click on the 2nd article title....,1652721816735,1652721833975,17240,0,False,"[{'userID': 30, 'frameNum': 1, 'unixTime': 165...",1652721816794,1652721833985,1,17191


In [41]:
groupA


,userID,task0,task1,task2,task3,task4,task5,task6,task7,task8,all_tasks,group
140,36,3250,7150,9881,5910,13390,12400,17100,17020,10820,96921,A
30,32,3994,4471,2915,7051,16741,28440,10321,39910,7950,121793,A
230,70,4574,10060,12190,6031,25960,15810,18989,27610,10460,131684,A
70,58,4340,11560,11071,16220,22540,38590,20421,20420,10928,156090,A
170,30,17191,12730,17180,9970,21701,16280,16921,41531,9373,162877,A
20,68,5242,8141,15202,4435,14831,13650,32120,38646,49680,181947,A
270,78,9690,8510,27099,5949,32371,33200,25280,21980,28561,192640,A
120,44,5831,8560,3060,10120,35321,12710,48740,34145,67541,226028,A
50,56,4110,33331,21496,10404,24967,19651,14430,148363,38121,314873,A
200,46,16441,49541,3001,20655,39420,28199,14470,140108,14528,326363,A


In [72]:
surveyDf = pandas.read_csv("PostStudy.csv")
surveyDf["group"] = surveyDf["What is your Participant Id?"].to_numpy() % 2
surveyDf

surveyDf = surveyDf.drop_duplicates(subset=['What is your Participant Id?'])
surveyDf

,Timestamp,What is your Participant Id?,Describe the task you found the easiest? Why?,Describe the task you found the most difficult? Why?,Did you find anything confusing about the interface?,"When the application was small, was all the content easy to read? Please explain",Did you notice any changes in the application as the size of the application changed?,"If yes, please explain what you noticed.",What would you change about the experiment?,It was easy to read with the applications.,It was easy to interact with the applications.,It was enjoyable to interact with the applications.,The tasks were easy to understand,It was easy to complete the tasks.,The content of the applications was easy to see.,Less application content would make the tasks easier.,I enjoyed the overall experience in terms of enjoyment.,The augmented reality (AR) headset was comfortable to wear.,I experienced fatigue during the experience.,group
0,2022/05/16 11:35:53 AM MDT,30,Click 2nd paper tile. Easy to click.,Type Isla Vista. Virtual keyboard is not easy.,"The scroll bar, should I press? Should I drag?",No. Get too close and text will be blurry again.,Yes,The weather window is very small.,It would be nice to avoid the scroll bar.,1,2,1,1,1,1,1,1,1,7,0
1,2022/05/16 12:45:28 PM MDT,31,"clicking on the links, they were pretty big an...",the keyboard typing,no. not really,"yes, but maybe not for everyone.",Yes,"there was more detail as I came closer, like l...","Nothing, it was really cool and fun.",2,2,3,2,2,2,6,2,2,6,1
2,2022/05/16 3:45:14 PM MDT,32,"Selecting titles/author names, they were clear...","The weather-related tasks, the text was very s...",Searching for information in the smaller displ...,"No, when the application was small the content...",No,NaN,"I don't have any suggestions, it was an intere...",5,5,4,6,6,4,5,6,6,7,0
3,2022/05/16 5:27:07 PM MDT,33,It was easiest to click on the title of things,using the keyboard was super annoying,"Nope, it seems straight forward","No, it was super far and I had to go super clo...",Yes,the temperature app became smaller,"I did not like typing on the keyboard in VR, i...",2,4,4,1,1,1,1,1,1,1,1
4,2022/05/16 6:29:53 PM MDT,34,The second time in Mode A was the easiest. I r...,The first time (in Mode A) was the hardest. I ...,No.,No. I could only distinguish letters within te...,No,NaN,Have it be stated that each code is split up a...,3,4,2,1,3,3,4,2,2,7,0
6,2022/05/17 11:20:33 AM MDT,35,"I don't remember the exact task name, there we...",Typing isla vista was very difficult. It was h...,I thought '<' represented delete but it was to...,"No, I had to move a little closer.",No,NaN,The buttons are very hard to press. Typing on ...,2,4,4,1,1,3,5,5,3,6,1
7,2022/05/17 12:47:50 PM MDT,36,Clicking on the article title was easiest beca...,Using the number keypad on the keyboard was th...,The scrollbar feels a bit clunky... as if it's...,"When the application was small, like the tempe...",No,NaN,Nothing I can think of.,2,3,5,1,3,5,5,5,3,3,0
8,2022/05/17 2:19:27 PM MDT,37,clicking the articles,i guess trying to pin the right letter or numb...,not really,"yes, everything was clear and legible",Yes,they were better calibrated when the app was s...,nothing!,2,1,2,1,1,1,4,1,1,4,1
9,2022/05/17 5:22:37 PM MDT,39,selecting an article. needed only a click,entering text. It is hard to press letters,No,No. I had to get really close to them,No,NaN,I would reduce the lag in the VR,3,6,3,2,3,3,2,6,6,1,1
10,2022/05/17 7:34:07 PM MDT,40,entering numbers,type a location. I made a mistake and backspac...,no,not really .,No,NaN,improving keyboard. I found it hard to work with,6,4,4,1,3,6,4,3,5,7,0


In [73]:
groupA = surveyDf.loc[surveyDf["group"] == 0]
groupA = groupA[groupA.columns[9]]

groupA

0     1
2     5
4     3
7     2
10    6
11    6
12    3
16    6
18    1
21    1
22    7
23    6
24    1
Name: It was easy to read with the applications., dtype: int64

In [74]:
for i in range(9,19):
    groupA = surveyDf.loc[surveyDf["group"] == 0]
    groupB = surveyDf.loc[surveyDf["group"] == 1]

    groupAresponseDf = groupA[groupA.columns[i]]
    groupBresponseDf = groupB[groupB.columns[i]]

    groupBResponse = groupBresponseDf.mean()
    groupBStd = groupBresponseDf.std()
    print(groupA.columns[i])
    print(f"group B {str(i)}: {groupBResponse:.3f} ± {groupBStd:.3f}")

    groupAResponse = groupAresponseDf.mean()
    groupAStd = groupAresponseDf.std()
    print(f"group A {str(i)}: {groupAResponse:.3f} ± {groupAStd:.3f}")

    print(scipy.stats.ttest_ind(groupBresponseDf, groupAresponseDf))

It was easy to read with the applications.
group B 9: 2.800 ± 1.424
group A 9: 3.692 ± 2.359
Ttest_indResult(statistic=-1.230810865200018, pvalue=0.22941242755724953)
It was easy to interact with the applications.
group B 10: 3.400 ± 1.549
group A 10: 3.923 ± 1.977
Ttest_indResult(statistic=-0.7843869210142275, pvalue=0.4399015774490348)
It was enjoyable to interact with the applications.
group B 11: 3.333 ± 1.234
group A 11: 3.462 ± 1.854
Ttest_indResult(statistic=-0.21810606963411452, pvalue=0.8290492739769972)
The tasks were easy to understand
group B 12: 2.000 ± 1.512
group A 12: 2.077 ± 2.019
Ttest_indResult(statistic=-0.11506557656088327, pvalue=0.9092771984159025)
It was easy to complete the tasks.
group B 13: 2.533 ± 1.598
group A 13: 3.000 ± 1.958
Ttest_indResult(statistic=-0.6945957783266239, pvalue=0.493469430503237)
The content of the applications was easy to see.
group B 14: 2.733 ± 1.710
group A 14: 3.077 ± 2.060
Ttest_indResult(statistic=-0.4824034673801531, pvalue=0.633